In [1]:
import pandas as pd
from utils import home, join, get_files, read_metadata_without_fields, read_metadata_with_fields
import numpy as np

In [2]:
rna_seq = pd.read_excel('pdxe_mol_data.xlsx', sheet_name='RNAseq_fpkm')
cna = pd.read_excel('pdxe_mol_data.xlsx', sheet_name='copy number')

In [80]:
dfci = join(home, 'DFCI-CPDM')
files = [f for f in get_files(dfci) if f.endswith('tsv')]
ps = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient_sample.tsv'))
pdx = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-pdx_model.tsv'))
mms = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_molecular_metadata-sample.tsv'))
mms_header = read_metadata_with_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_molecular_metadata-sample.tsv')).iloc[0:4,:]

expression_template = read_metadata_with_fields('expression_template-sheet.tsv')
samples = 'NIBR' + rna_seq.columns
matched_samples = rna_seq.columns[[True if s in ps.model_id.to_list() or s == 'NIBRSample' else False for s in samples]]
df = rna_seq[matched_samples]

for sample in df.columns[1:]:
    out_df = pd.DataFrame(columns=expression_template.columns)
    sample_id = 'NIBR' + sample.replace('X', '')
    out_df['rnaseq_fpkm'] = df[sample]
    out_df['symbol'] = df['Sample']
    out_df['sample_id'] = sample_id
    out_df['platform_id'] = 'expression_rna_seq'
    sum_fpkm = out_df['rnaseq_fpkm'].sum()
    out_df['rnaseq_tpm'] = (out_df['rnaseq_fpkm'] * 1e6) / sum_fpkm
    pdx_row = pdx[pdx['model_id'] == 'NIBR' + sample].reset_index(drop=True)
    mms_row = [['NIBR' + sample, sample_id, 'xenograft', pdx_row['passage_number'][0], pdx_row['host_strain_name'][0], pdx_row['host_strain_nomenclature'][0], pdx_row['engraftment_site'][0], '', 'expression_rna_seq']]
    mms = pd.concat([mms, pd.DataFrame(mms_row, columns=mms.columns)])
    out_df.to_csv(f'expression/DFCI-CPDM_expression_NIBR{sample}.tsv', sep='\t', index=False)
mms['Field'] = ''
mms = pd.concat([mms_header, mms])

In [79]:
mms.to_csv(join(home, 'DFCI-CPDM/DFCI-CPDM_molecular_metadata-sample.tsv'), sep='\t', index=False)

In [7]:
cna.iloc[:,1:].max().max()

173.65

In [8]:
cna.iloc[:,1:].min().min()

0.02

In [10]:
cna.iloc[:,1:].min().max()

0.43

In [14]:
cna.max() [cna.max() == 173.65]

X-4132    173.65
dtype: object

In [316]:
dfci = join(home, 'DFCI-CPDM')
files = [f for f in get_files(dfci) if f.endswith('tsv')]
ps_head = read_metadata_with_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient_sample.tsv')).iloc[0:4]
pat_head = read_metadata_with_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient.tsv')).iloc[0:4]
pdx_head = read_metadata_with_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-pdx_model.tsv')).iloc[0:4]
ps = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient_sample.tsv'))
pat = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient.tsv'))
pdx = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-pdx_model.tsv'))
dfci_cbio = read_metadata_with_fields("all_models_together_re_clinical_data.tsv")
matched_dfci = dfci_cbio[[True if mid in ps['model_id'].to_list() else False for mid in dfci_cbio['MODEL_ORIGINATOR_ID']]]

def match_model_id_with_dfci(mid, dfci_list, dfci):
    if mid in dfci_list:
        return mid
    if mid.__contains__('_'):
        if mid.__contains__('BT'):
            mid = mid.split('_')[0]
            if mid in dfci_list:
                return mid
        else:
            mid = '_'.join(mid.split('_')[0:2])
            if mid in dfci_list:
                return mid
    if mid.__contains__('-'):
        mid = '-'.join(mid.split('-')[0:2])
        if mid in dfci_list:
            return mid
    for _, row in dfci.iterrows():
        if row['SAMPLE_ID_LEGACY'].__contains__(mid):
            if not row['MODEL_ORIGINATOR_ID'].__contains__(' '):
                return row['MODEL_ORIGINATOR_ID']
    return mid



ps['matched_mid'] = ps['model_id'].apply(lambda x: match_model_id_with_dfci(x, dfci_cbio['MODEL_ORIGINATOR_ID'].to_list(), dfci_cbio))

matched_ps = ps[[True if mid in dfci_cbio['MODEL_ORIGINATOR_ID'].to_list() else False for mid in ps['matched_mid']]]

merged_df = matched_ps[['patient_id', 'sample_id', 'model_id', 'diagnosis', 'matched_mid']].merge(dfci_cbio[['Patient ID', 'MODEL_ORIGINATOR_ID','PATIENT_AGE_AT_SURGERY', 'RACE', 'PT_GENDER', 'PATIENT_TREATMENT_HISTORY', 'ETHNICITY']], left_on='matched_mid', right_on='MODEL_ORIGINATOR_ID', how='left').merge(pat, on='patient_id', how='left').drop_duplicates(subset=['model_id'])
model_ids_dfci = list(merged_df['model_id'].unique())

patient_id_mapper = {}
for _, row in ps.iterrows():
    if row['model_id'] in model_ids_dfci:
        match = merged_df[merged_df['matched_mid'] == row['matched_mid']].reset_index(drop=True)
        #print(f"{match['patient_id'][0]}, {match['Patient ID'][0]}")
        ethnicity = match['RACE'][0] if match['RACE'][0] != 'Unknown' else match['ETHNICITY'][0]
        #if row['patient_id'] not in patient_id_mapper.keys():
        patient_id_mapper[row['patient_id']] = [match['Patient ID'][0], match['PT_GENDER'][0], ethnicity, match['PATIENT_TREATMENT_HISTORY'][0]]
'''
        else:
            if match['Patient ID'][0] == patient_id_mapper[row['patient_id']][0]:
                continue
            else:
                print(f"{row['patient_id']}, {row['model_id']}")
                print(patient_id_mapper[row['patient_id']])
                print([match['Patient ID'][0], match['PT_GENDER'][0], ethnicity, match['PATIENT_TREATMENT_HISTORY'][0]])
        #ps.at[_, 'patient_id'] = match['Patient ID'][0]
'''
ps['Field'] = ''
out_df = pd.concat([ps_head, ps])
for _, row in pat.iterrows():
    if row['patient_id'] in patient_id_mapper.keys():
        pat.at[_, 'sex'] = patient_id_mapper[row['patient_id']][1].replace('Unknown', 'Not provided')
        pat.at[_, 'history'] = patient_id_mapper[row['patient_id']][3]
        pat.at[_, 'ethnicity'] = patient_id_mapper[row['patient_id']][2].replace('Unknown', 'Not provided').replace('Not Assessed', 'Not provided')
        #pat.at[_, 'patient_id'] = patient_id_mapper[row['patient_id']][0]
pat['Field'] = ''
out_pat_df = pd.concat([pat_head, pat])

In [317]:
out_df[ps_head.columns].to_csv(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient_sample.tsv'), sep='\t', index=False)

In [318]:
out_pat_df[pat_head.columns].to_csv(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient.tsv'), sep='\t', index=False)

In [319]:
ps = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient_sample.tsv'))
pat = read_metadata_without_fields(join(home, 'DFCI-CPDM/DFCI-CPDM_metadata-patient.tsv'))

merged_df = ps[['patient_id', 'model_id', 'diagnosis']].merge(pat[['patient_id', 'sex']], on='patient_id', how='left')
merged_df

,patient_id,model_id,diagnosis,sex
0,18-573-584,BT112_PDX_001,Glioblastoma NOS 4,Male
1,18-573-810,BT1150_PDX_001,Metastasis Lung,Female
2,18-573-585,BT116_PDX_001,Glioblastoma NOS 4,Female
3,18-573-349,BT127_PDX_001,Medulloblastoma 4,Male
4,18-573-731,BT188_PDX_001,Glioblastoma NOS 4,Female
...,...,...,...,...
730,18-573-1683,NIBRX-5974,Metastasis Breast,Female
731,18-573-1601,NIBRX-5975,Invasive Ductal Carcinoma,Female
732,18-573-1602,NIBRX-6017,Invasive Ductal Carcinoma,Female
733,18-573-1603,NIBRX-6030,Ductal Carcinoma,Male


In [275]:
ps[[True if pid not in patient_id_mapper.keys() else False for pid in ps.patient_id]]

,patient_id,sample_id,collection_date,collection_event,months_since_collection_1,age_in_years_at_collection,diagnosis,tumour_type,primary_site,collection_site,...,grade,grading_system,virology_status,gene_mutation_status,sharable,treatment_naive_at_collection,treated_at_collection,treated_prior_to_collection,response_to_treatment,model_id
0,CPDM_1272X,BT112,Not provided,Not provided,Not provided,59,Glioblastoma NOS 4,Primary,Frontal Cortex,Frontal Cortex,...,4,Not provided,Not provided,Not provided,Yes,yes,Not provided,No,Not provided,BT112_PDX_001
1,18-573-810,BT1150,Not provided,Not provided,Not provided,63,Metastasis Lung,Metastatic,Lung,Cerebellum,...,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,BT1150_PDX_001
2,18-573-585,BT116,Not provided,Not provided,Not provided,37,Glioblastoma NOS 4,Recurrent,Parietal Lobe,Parietal Lobe,...,4,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,BT116_PDX_001
3,18-573-349,BT127,Not provided,Not provided,Not provided,6,Medulloblastoma 4,Primary,Cerebellum,Cerebellum,...,4,Not provided,Not provided,Not provided,Not provided,yes,Not provided,Yes,Not provided,BT127_PDX_001
4,CPDM_1459X,BT188,Not provided,Not provided,Not provided,51,Glioblastoma NOS 4,Primary,Frontal Cortex,Frontal Cortex,...,4,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,Not provided,BT188_PDX_001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,18-573-1407,NIBR-0992,Not provided,Not provided,Not provided,77,Adenocarcinoma,Not provided,large intestine,Not Provided,...,Not provided,Not provided,Not provided,Not provided,No,Not provided,Not provided,Not provided,Not provided,NIBRX-0992
437,18-573-1519,NIBR-20689,Not provided,Not provided,Not provided,70,Squamous Cell Carcinoma,Not provided,lung,Lung,...,Not provided,Not provided,Not provided,Not provided,No,Not provided,Not provided,Not provided,Not provided,NIBRX-20689
572,18-573-1556,NIBR-3711,Not provided,Not provided,Not provided,61,Adenocarcinoma,Not provided,pancreas,Pancreas,...,Not provided,Not provided,Not provided,Not provided,No,Not provided,Not provided,Not provided,Not provided,NIBRX-3711
625,18-573-1615,NIBR-4377,Not provided,Not provided,Not provided,67,Ductal Adenocarcinoma,Metastatic,pancreas,Pancreas,...,2,Not provided,Not provided,Not provided,No,Not provided,Not provided,Not provided,Not provided,NIBRX-4377


In [85]:
merged_df[merged_df['diagnosis'] == 'Breast Cancer']

,patient_id,model_id,diagnosis,sex
374,CPDM_0231X,NIBRX-1371,Breast Cancer,Female


In [88]:
cm_si = pd.read_json("https://www.cancermodels.org/api/search_index?&patient_sex=in.(%22male%22)&search_terms=ov.{%22Breast%20Cancer%22}&select=provider_name,patient_age,patient_sex,external_model_id,model_type,data_source,histology,primary_site,collection_site,tumour_type,dataset_available,scores,model_availability_boolean&order=scores-%3E%3Edata_score.desc.nullslast,scores-%3E%3Epdx_metadata_score.desc.nullslast,scores-%3E%3Ein_vitro_metadata_score.desc.nullslast")

In [320]:
cm_si[['external_model_id', 'histology', 'data_source', 'patient_sex']].merge(merged_df, left_on='external_model_id', right_on='model_id', how='left')

,external_model_id,histology,data_source,patient_sex,patient_id,model_id,diagnosis,sex
0,NIBRX-3468,Malignant Breast Neoplasm,DFCI-CPDM,male,18-573-1547,NIBRX-3468,Metastasis Breast,Female
1,NIBRX-5355,Invasive Breast Carcinoma of No Special Type,DFCI-CPDM,male,18-573-1590,NIBRX-5355,Invasive Mammary Ductal Carcinoma,Female
2,NIBRX-3201,Malignant Breast Neoplasm,DFCI-CPDM,male,18-573-1537,NIBRX-3201,Metastasis Breast,Female
3,NIBRX-1916,Malignant Breast Neoplasm,DFCI-CPDM,male,18-573-1439,NIBRX-1916,Metastasis Breast,Female
4,NIBRX-1008,Invasive Breast Carcinoma of No Special Type,DFCI-CPDM,male,18-573-1321,NIBRX-1008,Invasive Ductal Carcinoma,Female
5,NIBRX-1298,Breast Ductal Carcinoma,DFCI-CPDM,male,18-573-1331,NIBRX-1298,Ductal Carcinoma,Female
6,NIBRX-2127,Breast Ductal Carcinoma,DFCI-CPDM,male,18-573-1366,NIBRX-2127,Ductal Carcinoma,Female
7,NIBRX-4888,Invasive Breast Carcinoma of No Special Type,DFCI-CPDM,male,18-573-1587,NIBRX-4888,Invasive Mammary Ductal Carcinoma,Female
8,NIBRX-3450,Invasive Breast Carcinoma of No Special Type,DFCI-CPDM,male,18-573-1545,NIBRX-3450,Invasive Ductal Carcinoma,Female
9,NIBRX-2780,Invasive Breast Carcinoma of No Special Type,DFCI-CPDM,male,18-573-1298,NIBRX-2780,Invasive Ductal Carcinoma,Female


In [96]:
cm_si_oc = pd.read_json("https://www.cancermodels.org/api/search_index?&patient_sex=in.(%22male%22)&search_terms=ov.{%22Ovarian%20Cancer%22}&select=provider_name,patient_age,patient_sex,external_model_id,model_type,data_source,histology,primary_site,collection_site,tumour_type,dataset_available,scores,model_availability_boolean&order=scores-%3E%3Edata_score.desc.nullslast,scores-%3E%3Epdx_metadata_score.desc.nullslast,scores-%3E%3Ein_vitro_metadata_score.desc.nullslast")

In [321]:
cm_si_oc[['external_model_id', 'histology', 'data_source', 'patient_sex']].merge(merged_df, left_on='external_model_id', right_on='model_id', how='left')

,external_model_id,histology,data_source,patient_sex,patient_id,model_id,diagnosis,sex
0,NIBRX-3615,Malignant Ovarian Neoplasm,DFCI-CPDM,male,18-573-1550,NIBRX-3615,Metastasis Ovary,Female
1,NIBRX-1786,Ovarian High Grade Serous Adenocarcinoma,DFCI-CPDM,male,18-573-1510,NIBRX-1786,High Grade Serous Carcinoma,Female
2,NIBRX-3604,Malignant Ovarian Neoplasm,DFCI-CPDM,male,18-573-1490,NIBRX-3604,Metastasis Ovary,Female
3,NIBRX-3694,Ovarian Serous Adenocarcinoma,DFCI-CPDM,male,18-573-1554,NIBRX-3694,Serous Adenocarcinoma,Female
4,NIBRX-1318,Malignant Ovarian Neoplasm,DFCI-CPDM,male,18-573-1418,NIBRX-1318,Metastasis Ovary,Female
5,NIBRX-2805,Ovarian High Grade Serous Adenocarcinoma,DFCI-CPDM,male,18-573-1469,NIBRX-2805,High Grade Serous Carcinoma,Female
6,NIBRX-2471,Ovarian High Grade Serous Adenocarcinoma,DFCI-CPDM,male,18-573-1530,NIBRX-2471,High Grade Serous Carcinoma,Female
7,NIBRX-4261,Ovarian High Grade Serous Adenocarcinoma,DFCI-CPDM,male,18-573-1306,NIBRX-4261,High Grade Serous Carcinoma,Female
8,NIBRX-4829,Ovarian Serous Adenocarcinoma,DFCI-CPDM,male,18-573-1309,NIBRX-4829,Serous Carcinoma,Female
9,NIBRX-3575,Ovarian High Grade Serous Adenocarcinoma,DFCI-CPDM,male,18-573-1609,NIBRX-3575,High Grade Serous Carcinoma,Female
